In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/

!git clone https://github.com/ztuntrade/untrade-sdk.git
%cd untrade-sdk
!pip3 install .
from untrade.client import Client
client = Client()
!pip install empyrical


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4063    0  4063    0     0  14122      0 --:--:-- --:--:-- --:--:-- 14156
100  517k  100  517k    0     0   493k      0  0:00:01  0:00:01 --:--:-- 1657k
--2025-04-14 09:21:00--  https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
Resolving anaconda.org (anaconda.org)... 104.19.144.37, 104.19.145.37, 2606:4700::6813:9125, ...
Connecting to anaconda.org (anaconda.org)|104.19.144.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5f738b4b78d4370a69f82984/673b62920c609ff6945bb24d?response-content-disposition=attachment%3B%20filename%3D%22ta-lib-0.5.1-py311h9ecbd09_0.conda%22%3B%20filename%2A%3DUTF-8%27%27ta-lib-0.5.1-py311h9ecbd09_0.conda&response-content-type=application%2Foctet-stream&X-Amz-Algorithm=

In [11]:
import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import os
import uuid
from untrade.client import Client
import talib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from empyrical import max_drawdown
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import empyrical
from scipy import stats
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [12]:
def process_data(data_path):
    """Process the data and calculate technical indicators"""
    data = pd.read_csv("/content/drive/MyDrive/BTC_2019_2023_1h.csv")

    # Calculate technical indicators
    data['hammer'] = talib.CDLHAMMER(data.open, data.high, data.low, data.close)
    data['hanging_man'] = talib.CDLHANGINGMAN(data.open, data.high, data.low, data.close)
    data['doji'] = talib.CDLDOJI(data.open, data.high, data.low, data.close)
    data['spinning_top'] = talib.CDLSPINNINGTOP(data.open, data.high, data.low, data.close)
    data['engulfing'] = talib.CDLENGULFING(data.open, data.high, data.low, data.close)
    data['RSI'] = talib.RSI(data.close, timeperiod=14)
    data['slowk'], data['slowd'] = talib.STOCH(data.high, data.low, data.close, fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    data['sma_10'] = data['close'].rolling(window=10).mean()
    data['sma_05'] = data['close'].rolling(window=5).mean()
    data['sma_20'] = data['close'].rolling(window=20).mean()
    data['sma_30'] = data['close'].rolling(window=30).mean()
    data['sma_50'] = data['close'].rolling(window=50).mean()
    data['sma_200'] = data['close'].rolling(window=200).mean()
    data['sma_600'] = data['close'].rolling(window=600).mean()
    data['sma_40'] = data['close'].rolling(window=40).mean()
    data['sma_120'] = data['close'].rolling(window=120).mean()
    data['ema_05'] = talib.EMA(data['close'], timeperiod=5)
    data['ema_10'] = talib.EMA(data['close'], timeperiod=10)
    data['ema_30'] = talib.EMA(data['close'], timeperiod=30)
    data['MACD'], _, _ = talib.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    data['typical_price'] = (data['high'] + data['low'] + data['close']) / 3
    data['cumulative_price_volume'] = data['typical_price'] * data['volume']
    data['cumulative_volume'] = data['volume'].cumsum()
    data['cumulative_price'] = data['cumulative_price_volume'].cumsum()
    data['vwap'] = data['cumulative_price'] / data['cumulative_volume']

    # Additional features
    data['Stochastic Oscillator'] = talib.STOCH(data['high'], data['low'], data['close'])[1]
    data['Average True Range'] = talib.ATR(data['high'], data['low'], data['close'], timeperiod=14)
    data['Bollinger Bands Width'] = (talib.BBANDS(data['close'])[0] - talib.BBANDS(data['close'])[2]) / talib.BBANDS(data['close'])[1]

    # Price returns and momentum
    data['diff'] = data['close'].diff()
    data['Price_Ret'] = data['close'].pct_change()
    data['Price_Momentum'] = data['Price_Ret'].rolling(window=5).mean()

    # Find local maxima and minima
    window_size = 24
    local_maxima_indices = argrelextrema(data['close'].values, np.greater, order=window_size)[0]
    local_minima_indices = argrelextrema(data['close'].values, np.less, order=window_size)[0]

    # Mark local extrema in DataFrame
    data['is_maxima'] = 0
    data.loc[local_maxima_indices, 'is_maxima'] = 1
    data['is_minima'] = 0
    data.loc[local_minima_indices, 'is_minima'] = 1

    # Create binary labels for price direction
    data['Price_Up'] = (data['Price_Ret'] > 0).astype(int)
    data['Price_Up'] = data['Price_Up'].shift(-1)

    return data

def train_ml_models(data):
    """Train machine learning models for extrema detection"""
    # Clean up data for model training
    cp = data.copy()
    cp.dropna(inplace=True)

    # Define features
    features = ['RSI', 'ema_10', 'MACD', 'Price_Momentum']
    X_rf = cp[features]

    # Additional features for decision tree
    add_features = ['Stochastic Oscillator', 'Average True Range',
                   'Bollinger Bands Width', 'volume', 'slowk']
    X_dt = cp[features + add_features]

    # Train Random Forest minima classifier
    y = cp['is_minima']
    X_train, X_test, y_train, y_test = train_test_split(X_rf, y, test_size=0.3, random_state=42)
    rf_minima_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_minima_classifier.fit(X_train, y_train)

    # Train Random Forest maxima classifier
    y = cp['is_maxima']
    X_train, X_test, y_train, y_test = train_test_split(X_rf, y, test_size=0.3, random_state=42)
    rf_maxima_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_maxima_classifier.fit(X_train, y_train)

    # Train Decision Tree minima classifier
    y = cp['is_minima']
    X_train, X_test, y_train, y_test = train_test_split(X_dt, y, test_size=0.3, random_state=42)
    dt_minima_classifier = DecisionTreeClassifier(random_state=42)
    dt_minima_classifier.fit(X_train, y_train)

    # Train Decision Tree maxima classifier
    y = cp['is_maxima']
    X_train, X_test, y_train, y_test = train_test_split(X_dt, y, test_size=0.3, random_state=42)
    dt_maxima_classifier = DecisionTreeClassifier(random_state=42)
    dt_maxima_classifier.fit(X_train, y_train)

    return {
        'rf_minima': rf_minima_classifier,
        'rf_maxima': rf_maxima_classifier,
        'dt_minima': dt_minima_classifier,
        'dt_maxima': dt_maxima_classifier,
        'features': features,
        'add_features': add_features
    }

In [13]:
def implement_strategy(data, models, nn_model):
    """Implement the trading strategy and generate signals"""
    features = models['features']
    add_features = models['add_features']
    rf_minima_classifier = models['rf_minima']
    rf_maxima_classifier = models['rf_maxima']
    dt_minima_classifier = models['dt_minima']
    dt_maxima_classifier = models['dt_maxima']

    # Initialize signal columns
    data['signal'] = 0
    data['position'] = 0

    # Helper functions for classification
    def rf_minima_classify(i):
        if i >= len(data) or any(pd.isna(data[features].iloc[i])):
            return 0
        x_feat = data[features].iloc[[i]]
        y_pred = rf_minima_classifier.predict(x_feat)
        return y_pred[0]

    def rf_maxima_classify(i):
        if i >= len(data) or any(pd.isna(data[features].iloc[i])):
            return 0
        x_feat = data[features].iloc[[i]]
        y_pred = rf_maxima_classifier.predict(x_feat)
        return y_pred[0]

    def dt_minima_classify(i):
        if i >= len(data) or any(pd.isna(data[features + add_features].iloc[i])):
            return 0
        x_feat = data[features + add_features].iloc[[i]]
        y_pred = dt_minima_classifier.predict(x_feat)
        return y_pred[0]

    def dt_maxima_classify(i):
        if i >= len(data) or any(pd.isna(data[features + add_features].iloc[i])):
            return 0
        x_feat = data[features + add_features].iloc[[i]]
        y_pred = dt_maxima_classifier.predict(x_feat)
        return y_pred[0]

    # Helper functions for entry/exit
    def entry(i):
        if i < 10 or i >= len(data):
            return False
        if rf_minima_classify(i) == 1 and dt_minima_classify(i) == 1:
            last_10_close = data['diff'][i - 9 : i + 1]
            if len(last_10_close) == 10 and not any(pd.isna(last_10_close)):
                last_10_close = np.array(last_10_close)
                last_10_close = last_10_close.reshape(1, 10)
                output = nn_model.predict(last_10_close)
                if output[0][0] < output[0][1]:
                    return True
        return False

    def exit(i):
        if i < 10 or i >= len(data):
            return False
        if rf_maxima_classify(i) == 1 and dt_maxima_classify(i) == 1:
            last_10_close = data['diff'][i - 9 : i + 1]
            if len(last_10_close) == 10 and not any(pd.isna(last_10_close)):
                last_10_close = np.array(last_10_close)
                last_10_close = last_10_close.reshape(1, 10)
                output = nn_model.predict(last_10_close)
                if output[0][0] > output[0][1]:
                    return True
        return False

    # Apply strategy with stop loss
    max_drawdown_limit = 0.03
    returns = []
    position = 0
    in_trade = False
    short_trade = False
    buy_price = 0

    valid_range = range(10, min(30000, len(data) - 1))

    for i in valid_range:
        # Long entry
        if not in_trade and not short_trade and entry(i):
            buy_price = data['close'][i]
            data.at[i, 'signal'] = 1
            data.at[i, 'position'] = 1
            in_trade = True

        # Long exit
        elif in_trade and (exit(i) or data['close'][i] < buy_price * (1 - max_drawdown_limit)):
            pnl = (data['close'][i] - buy_price) / buy_price
            returns.append(pnl)
            data.at[i, 'signal'] = -1
            data.at[i, 'position'] = 0
            in_trade = False

        # Short entry
        elif not short_trade and not in_trade and exit(i):
            buy_price = data['close'][i]
            data.at[i, 'signal'] = -1
            data.at[i, 'position'] = -1
            short_trade = True

        # Short exit
        elif short_trade and (entry(i) or data['close'][i] > buy_price * (1 + max_drawdown_limit)):
            pnl = (buy_price - data['close'][i]) / buy_price
            returns.append(pnl)
            data.at[i, 'signal'] = 1
            data.at[i, 'position'] = 0
            short_trade = False

    # Close any remaining positions at the end
    if in_trade:
        i = valid_range[-1]
        data.at[i, 'signal'] = -1
        data.at[i, 'position'] = 0
        pnl = (data['close'][i] - buy_price) / buy_price
        returns.append(pnl)

    if short_trade:
        i = valid_range[-1]
        data.at[i, 'signal'] = 1
        data.at[i, 'position'] = 0
        pnl = (buy_price - data['close'][i]) / buy_price
        returns.append(pnl)

    # Calculate cumulative returns
    returns = np.array(returns)
    if len(returns) > 0:
        profit_trades = returns[returns > 0]
        loss_trades = returns[returns < 0]

        print(f"Profit trades: {len(profit_trades)}, Average profit: {profit_trades.mean() if len(profit_trades) > 0 else 0}")
        print(f"Loss trades: {len(loss_trades)}, Average loss: {loss_trades.mean() if len(loss_trades) > 0 else 0}")

        cum_pnl = 1 + np.sum(returns)
        mxd = max_drawdown(returns) if len(returns) > 0 else 0

        print('Drawdown: ', mxd)
        print('Total PNL: ', cum_pnl)
        print('Fees: ', len(returns) * 0.001)  # Assuming 0.1% fee per trade
        print('Return: ', ((cum_pnl - 1) * 100 - (len(returns) * 0.1)), '%')

    # Prepare final dataframe for untrade SDK
    result_df = data[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signal']].copy()
    result_df.rename(columns={'signal': 'signals'}, inplace=True)

    return result_df

def load_neural_network_model(model_path):
    """Load the neural network model for prediction"""
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(synchronized=True),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.BatchNormalization(synchronized=True),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(2)
    ])

    model.build(input_shape=(1, 10))
    model.load_weights(model_path)
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model



In [14]:
def perform_backtest(csv_file_path, jupyter_id="test"):
    """Perform backtesting using the untrade SDK"""
    client = Client()

    # Perform backtest using the provided CSV file path
    result = client.backtest(
        jupyter_id="test",  # Replace with your jupyter ID for untrade.io
        file_path=csv_file_path,
        leverage=1,
        # result_type="Q"
    )

    return result

def display_backtest_results(result):
    """Display full backtest results from untrade SDK"""
    for value in result:
        print(value)

    # The last value should contain the full backtest results
    result_list = list(result)
    if result_list:
        print("\nFull Portfolio Statistics:")
        final_result = result_list[-1]

        # Extract and print detailed metrics
        # The exact structure will depend on untrade SDK's output format
        if isinstance(final_result, dict):
            metrics = [
                'Total Return (%)', 'Ann. Return (%)', 'Ann. Volatility (%)',
                'Sharpe Ratio', 'Sortino Ratio', 'Max Drawdown (%)',
                'Avg Win (%)', 'Avg Loss (%)', 'Win Rate (%)',
                'Profit Factor', 'Recovery Factor', 'Calmar Ratio',
                'Number of Trades'
            ]

            print("\n===== Portfolio Performance Metrics =====")
            for metric in metrics:
                if metric in final_result:
                    print(f"{metric}: {final_result[metric]}")

            # Add additional sections based on what untrade SDK provides
            print("\n===== Trade Statistics =====")
            if 'long_trades' in final_result and 'short_trades' in final_result:
                print(f"Long Trades: {final_result['long_trades']}")
                print(f"Short Trades: {final_result['short_trades']}")

            print("\n===== Risk Metrics =====")
            if 'var' in final_result and 'cvar' in final_result:
                print(f"Value at Risk (95%): {final_result['var']}")
                print(f"Conditional VaR (95%): {final_result['cvar']}")


In [15]:
# Add this function to your existing code

def visualize_backtest_results(backtest_result, data):
    """Generate comprehensive graphs of portfolio statistics using untrade SDK results"""
    print("Generating portfolio performance visualizations...")

    # Create a directory for the graphs if it doesn't exist
    import os
    if not os.path.exists('strategy_graphs'):
        os.makedirs('strategy_graphs')

    # Convert the backtest results to a more usable format
    result_list = list(backtest_result)

    if not result_list:
        print("No backtest results to visualize.")
        return

    final_result = result_list[-1]

    # Extract portfolio equity curve and daily returns if available
    if 'equity_curve' in final_result:
        equity_curve = pd.DataFrame(final_result['equity_curve'])
        equity_curve['date'] = pd.to_datetime(equity_curve['date'])

        # Plot equity curve
        plt.figure(figsize=(12, 6))
        plt.plot(equity_curve['date'], equity_curve['equity'], label='Portfolio Value', color='blue')
        plt.title('Portfolio Equity Curve')
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.savefig('strategy_graphs/equity_curve.png')
        plt.close()

        # Plot drawdowns
        if 'drawdown' in equity_curve.columns:
            plt.figure(figsize=(12, 6))
            plt.fill_between(equity_curve['date'], 0, equity_curve['drawdown'] * 100, color='red', alpha=0.3)
            plt.plot(equity_curve['date'], equity_curve['drawdown'] * 100, color='red')
            plt.title('Portfolio Drawdown')
            plt.xlabel('Date')
            plt.ylabel('Drawdown (%)')
            plt.grid(True, alpha=0.3)
            plt.savefig('strategy_graphs/drawdown.png')
            plt.close()

    # Use the processed data to plot signals on the price chart
    data_with_signals = data.copy()
    data_with_signals['datetime'] = pd.to_datetime(data_with_signals['datetime'])

    # Plot price chart with buy/sell signals
    plt.figure(figsize=(14, 7))
    plt.plot(data_with_signals['datetime'], data_with_signals['close'], label='BTC/USDT', color='black', alpha=0.5)

    # Plot buy signals
    buy_signals = data_with_signals[data_with_signals['signals'] == 1]
    plt.scatter(buy_signals['datetime'], buy_signals['close'], marker='^', color='green', s=100, label='Buy Signal')

    # Plot sell signals
    sell_signals = data_with_signals[data_with_signals['signals'] == -1]
    plt.scatter(sell_signals['datetime'], sell_signals['close'], marker='v', color='red', s=100, label='Sell Signal')

    plt.title('BTC/USDT Price with Trading Signals')
    plt.xlabel('Date')
    plt.ylabel('Price (USDT)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.savefig('strategy_graphs/signals.png')
    plt.close()

    # If untrade SDK provides trade data, plot trade distribution
    if 'trades' in final_result:
        trades_df = pd.DataFrame(final_result['trades'])

        if not trades_df.empty and 'pnl' in trades_df.columns:
            # Plot trade PnL distribution
            plt.figure(figsize=(10, 6))
            sns.histplot(trades_df['pnl'], kde=True, bins=20)
            plt.axvline(x=0, color='r', linestyle='--')
            plt.title('Trade PnL Distribution')
            plt.xlabel('Profit/Loss')
            plt.ylabel('Frequency')
            plt.grid(True, alpha=0.3)
            plt.savefig('strategy_graphs/pnl_distribution.png')
            plt.close()

            # Plot cumulative trades over time
            if 'exit_time' in trades_df.columns:
                trades_df['exit_time'] = pd.to_datetime(trades_df['exit_time'])
                trades_df = trades_df.sort_values('exit_time')
                trades_df['cumulative_pnl'] = trades_df['pnl'].cumsum()

                plt.figure(figsize=(12, 6))
                plt.plot(trades_df['exit_time'], trades_df['cumulative_pnl'], label='Cumulative PnL')
                plt.title('Cumulative Trade Performance')
                plt.xlabel('Date')
                plt.ylabel('Cumulative PnL')
                plt.grid(True, alpha=0.3)
                plt.legend()
                plt.savefig('strategy_graphs/cumulative_pnl.png')
                plt.close()

    # Create a comprehensive performance dashboard using Plotly
    try:
        # Prepare data for dashboard
        if 'equity_curve' in final_result:
            fig = make_subplots(
                rows=3, cols=2,
                subplot_titles=(
                    'Portfolio Equity Curve', 'Drawdown',
                    'Monthly Returns Heatmap', 'Return Distribution',
                    'Trade Duration vs PnL', 'Win/Loss Ratio'
                ),
                specs=[
                    [{"type": "scatter"}, {"type": "scatter"}],
                    [{"type": "heatmap"}, {"type": "histogram"}],
                    [{"type": "scatter"}, {"type": "pie"}],
                ],
                vertical_spacing=0.1,
                horizontal_spacing=0.1
            )

            # Plot 1: Equity Curve
            fig.add_trace(
                go.Scatter(
                    x=equity_curve['date'],
                    y=equity_curve['equity'],
                    name='Portfolio Value',
                    line=dict(color='blue')
                ),
                row=1, col=1
            )

            # Plot 2: Drawdown
            if 'drawdown' in equity_curve.columns:
                fig.add_trace(
                    go.Scatter(
                        x=equity_curve['date'],
                        y=equity_curve['drawdown'] * 100,
                        name='Drawdown',
                        fill='tozeroy',
                        line=dict(color='red')
                    ),
                    row=1, col=2
                )

            # Plot 3: Monthly Returns Heatmap
            if 'returns' in equity_curve.columns:
                equity_curve['year'] = equity_curve['date'].dt.year
                equity_curve['month'] = equity_curve['date'].dt.month
                monthly_returns = equity_curve.groupby(['year', 'month'])['returns'].sum().unstack()

                month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

                fig.add_trace(
                    go.Heatmap(
                        z=monthly_returns.values,
                        x=month_names[:monthly_returns.shape[1]],
                        y=monthly_returns.index,
                        colorscale='RdBu',
                        zmid=0
                    ),
                    row=2, col=1
                )

            # Plot 4: Return Distribution
            if 'returns' in equity_curve.columns:
                fig.add_trace(
                    go.Histogram(
                        x=equity_curve['returns'],
                        nbinsx=30,
                        marker_color='blue',
                        opacity=0.7
                    ),
                    row=2, col=2
                )

            # Plot 5: Trade Duration vs PnL
            if 'trades' in final_result and 'duration' in trades_df.columns and 'pnl' in trades_df.columns:
                fig.add_trace(
                    go.Scatter(
                        x=trades_df['duration'],
                        y=trades_df['pnl'],
                        mode='markers',
                        marker=dict(
                            size=8,
                            color=trades_df['pnl'],
                            colorscale='RdBu',
                            line=dict(width=1),
                            cmid=0
                        )
                    ),
                    row=3, col=1
                )

            # Plot 6: Win/Loss Ratio Pie Chart
            if 'trades' in final_result and 'pnl' in trades_df.columns:
                wins = (trades_df['pnl'] > 0).sum()
                losses = (trades_df['pnl'] < 0).sum()
                breakeven = (trades_df['pnl'] == 0).sum()

                fig.add_trace(
                    go.Pie(
                        labels=['Wins', 'Losses', 'Breakeven'],
                        values=[wins, losses, breakeven],
                        textinfo='label+percent',
                        hole=.3,
                        marker_colors=['green', 'red', 'gray']
                    ),
                    row=3, col=2
                )

            # Update layout
            fig.update_layout(
                title_text='Trading Strategy Performance Dashboard',
                height=900,
                width=1200,
                showlegend=False
            )

            # Save as interactive HTML
            fig.write_html('strategy_graphs/performance_dashboard.html')

            # Create a static version for image output
            fig.write_image('strategy_graphs/performance_dashboard.png')

        print("Visualization complete. Graphs saved to 'strategy_graphs' directory.")

    except Exception as e:
        print(f"Error creating performance dashboard: {e}")
        print("Continuing with other visualizations...")

    # Generate additional performance metrics and save to CSV
    try:
        if 'equity_curve' in final_result:
            # Calculate additional performance metrics
            metrics_df = pd.DataFrame({
                'Metric': [
                    'Total Return (%)',
                    'Annualized Return (%)',
                    'Annualized Volatility (%)',
                    'Sharpe Ratio',
                    'Sortino Ratio',
                    'Max Drawdown (%)',
                    'Win Rate (%)',
                    'Profit Factor',
                    'Average Win (%)',
                    'Average Loss (%)',
                    'Largest Win (%)',
                    'Largest Loss (%)',
                    'Average Trade Duration (hours)',
                    'Total Number of Trades'
                ],
                'Value': [
                    f"{final_result.get('total_return', 0) * 100:.2f}",
                    f"{final_result.get('annual_return', 0) * 100:.2f}",
                    f"{final_result.get('annual_volatility', 0) * 100:.2f}",
                    f"{final_result.get('sharpe_ratio', 0):.2f}",
                    f"{final_result.get('sortino_ratio', 0):.2f}",
                    f"{final_result.get('max_drawdown', 0) * 100:.2f}",
                    f"{final_result.get('win_rate', 0) * 100:.2f}",
                    f"{final_result.get('profit_factor', 0):.2f}",
                    f"{final_result.get('avg_win', 0) * 100:.2f}",
                    f"{final_result.get('avg_loss', 0) * 100:.2f}",
                    f"{final_result.get('largest_win', 0) * 100:.2f}",
                    f"{final_result.get('largest_loss', 0) * 100:.2f}",
                    f"{final_result.get('avg_trade_duration', 0):.2f}",
                    f"{final_result.get('total_trades', 0)}"
                ]
            })

            # Save to CSV
            metrics_df.to_csv('strategy_graphs/performance_metrics.csv', index=False)

            # Print summary
            print("\nPerformance Metrics Summary:")
            print(metrics_df.to_string(index=False))

    except Exception as e:
        print(f"Error generating performance metrics: {e}")

# Update your main function to include visualization
def main():
    # File paths
    btc_csv_path = "/content/drive/MyDrive/BTCUSDT_2019_2023_1h.csv"
    model_path = '/content/drive/MyDrive/modelweights_1hr_nn.h5'

    if not os.path.exists(btc_csv_path):
        print(f"Missing file: {btc_csv_path}")
        return

    if not os.path.exists(model_path):
        print(f"Missing neural network model: {model_path}")
        return

    # Load the neural network model
    nn_model = load_neural_network_model(model_path)

    # Process data and calculate indicators
    print("Processing data and calculating indicators...")
    data = process_data(btc_csv_path)

    # Train machine learning models
    print("Training machine learning models...")
    models = train_ml_models(data)

    # Implement trading strategy
    print("Implementing trading strategy...")
    result_df = implement_strategy(data, models, nn_model)

    # Save to CSV for backtesting
    output_csv_path = "btc_strategy_results.csv"
    result_df.to_csv(output_csv_path, index=False)

    # Perform backtest with untrade SDK
    print("Performing backtest with untrade SDK...")
    backtest_result = perform_backtest(output_csv_path, jupyter_id="test")

    # Display detailed backtest results
    print("Backtest Results:")
    display_backtest_results(backtest_result)

    # Visualize backtest results
    visualize_backtest_results(backtest_result, result_df)

    # Print summary of signals
    print("\nSignal Summary:")
    print(f"Total Buy Signals: {len(result_df[result_df['signals'] == 1])}")
    print(f"Total Sell Signals: {len(result_df[result_df['signals'] == -1])}")

    print("Strategy execution and visualization completed.")

# Add a direct untrade SDK graph function for additional visualization options
def generate_untrade_sdk_graphs(backtest_result, output_dir="strategy_graphs"):
    """
    Generate graphs directly using untrade SDK's built-in visualization capabilities
    """
    try:
        # Convert backtest result to the expected format
        result_list = list(backtest_result)
        if not result_list:
            print("No backtest results for untrade SDK graph generation")
            return

        result = result_list[-1]
        client = Client()

        # Ensure output directory exists
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Generate untrade SDK graphs - using their built-in visualization functions
        # The exact function names and parameters will depend on the untrade SDK
        # The following is just an example - adjust based on actual SDK documentation

        print("Generating untrade SDK visualizations...")

        # Example: Generate equity curve
        client.plot_equity_curve(result, save_path=f"{output_dir}/untrade_equity_curve.png")

        # Example: Generate drawdown chart
        client.plot_drawdown(result, save_path=f"{output_dir}/untrade_drawdown.png")

        # Example: Generate returns distribution
        client.plot_returns_distribution(result, save_path=f"{output_dir}/untrade_returns_dist.png")

        # Example: Generate trade analysis
        client.plot_trade_analysis(result, save_path=f"{output_dir}/untrade_trade_analysis.png")

        # Example: Generate rolling metrics
        client.plot_rolling_metrics(result, save_path=f"{output_dir}/untrade_rolling_metrics.png")

        # Example: Generate comprehensive dashboard
        client.generate_performance_report(result, save_path=f"{output_dir}/untrade_report.html")

        print(f"Untrade SDK visualizations saved to {output_dir} directory")

    except Exception as e:
        print(f"Error generating untrade SDK graphs: {e}")
        print("If these functions don't exist in the SDK, please refer to the SDK documentation")

# When running the main function, also generate untrade-specific graphs
if __name__ == "__main__":
    main()
    # Try to generate untrade SDK-specific graphs if available
    try:
        # Get the latest backtest result
        client = Client()
        latest_result = client.get_latest_backtest("test")  # Using the same jupyter_id as in backtest
        generate_untrade_sdk_graphs(latest_result)
    except Exception as e:
        print(f"Note: Could not generate untrade SDK-specific graphs: {e}")
        print("This may be due to SDK version differences or API changes.")

Processing data and calculating indicators...
Training machine learning models...
Implementing trading strategy...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━